In [1]:
import time
import requests, re
from bs4 import BeautifulSoup

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

from datetime import datetime, timedelta

from tqdm import tqdm

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
headers = {
    'user-agent' : ua.random
}
base_url = 'https://df.nexon.com/df/community/dnfboard'
params = {
    'mode': 'list',
    'order': 'reg_date',
    'order_type': 'DESC',
    'job' : '99', # 게시판 코드 99-> 공통
    "grow_type": '0',
    'page' : '0' # 페이지 설정
}

In [34]:
# 검색할때 설정해줘야할 것들
for target_date in dates:
    path = './data/offical_page'

    df = load_data(path, target_date)

    if not len(df):
        is_concat = False

    else:
        is_concat = True
        continue

    # dc_id = "dnfqq"

    # if len(df) == 0:
    #     y_day = datetime.today() - timedelta(days=1)
    #     y_day_string = str(y_day.year) + "-" + str(y_day.month) + "-" + str(y_day.day)
    #     last_timestamp = pd.to_datetime(y_day_string)
    # else:
    #     last_timestamp = pd.to_datetime(df.sort_values('timestamp', ascending=False)['timestamp'].iloc[0])

    # print(last_timestamp)

    search_data = search_post(pd.to_datetime(target_date))

    if is_concat:
        print('Concat New Post')
        df = pd.concat([pd.DataFrame(search_data), df])
    else:
        print('Frist Crawling')
        df = pd.DataFrame(search_data)

    save_daily_data(df, path, target_date)

20230515
./data/offical_page_20230515.csv
File exist Load FIN!!
20230511
./data/offical_page_20230511.csv
File exist Load FIN!!
20230512
./data/offical_page_20230512.csv
File exist Load FIN!!
20230513
./data/offical_page_20230513.csv
SEARCH POST
request Setting FIN
14 13 PASSING
request Setting FIN
HTTPSConnectionPool(host='df.nexon.com', port=443): Max retries exceeded with url: /df/community/dnfboard?mode=list&order=reg_date&order_type=DESC&job=99&grow_type=0&page=31 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out')))
request Setting FIN
14 13 PASSING
request Setting FIN
14 13 PASSING
request Setting FIN
14 13 PASSING
request Setting FIN
GET POST
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
34
request Setting FIN
GET POST
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
35
request Setting FIN
GET POST
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
PASSING NOTICE
PAS

In [33]:
def get_post(soup, target_date):
    print('GET POST')
    base_url = 'https://df.nexon.com/df/community/dnfboard'
    break_flag = False
    posts = soup.select('table.tbl > tbody > tr')
    
    cols = ['title', 'writed_at', 'refresh', 'recommand', 'link']
    
    post_dict = {key : [] for key in cols}
    
    notice_cnt = 0
    
    block_text = '신고에 의해 숨김 처리된 게시물입니다.'
    
    for post in posts:
        try:
            if post['class']: 
                print('PASSING NOTICE')
                notice_cnt+=1
                continue
        except:
            pass # 공지가 아닌 글들은 class가 없어서 오류 발생
        
        if post.select('td'):
            
            # 신고 처리된 게시물일 경우 패스
            if post.select_one('td.tit > strong'):
                print(post.select_one('td.tit > strong').text)
                notice_cnt+=1
                continue
            
            writed_at = post.select('td')[2].text
            
            # 검색 날짜보다 이전 결과라면 탈출
            if pd.to_datetime('2023-' + writed_at).day < target_date.day:
                print('break', pd.to_datetime('2023-' + writed_at))
                break
            
            post_info = post.select_one('td.tit > a')
            title = post_info['title']
            link = base_url + post_info['href']
#             title = re.sub('[\r\n]+', '' , title).strip()


            refresh = post.select('td')[3].text
            recommand = post.select('td')[4].text

            data = [title, writed_at, refresh, recommand, link]
            
            for key, value in zip(cols, data):
                post_dict[key].append(value)

    
    if len(posts) -1 - notice_cnt  > len(post_dict['title']):
        print('SEARCH FIN! BREAK ROOP')
        break_flag = True
    
    return pd.DataFrame(post_dict), break_flag

    
    

In [31]:
def search_post(target_date):
    print('SEARCH POST')
    
    df_lst = []
    
    headers = {
        'user-agent' : ua.random
    }
    url = 'https://df.nexon.com/df/community/dnfboard'
    params = {
        'mode': 'list',
        'order': 'reg_date',
        'order_type': 'DESC',
        'job' : '99', # 게시판 코드 99-> 공통
        "grow_type": '0',
    #     'page' : '1' # 페이지 설정
        
    }
    
    proxy_data = get_proxy_list()
    
    proxy_idx = 0
    break_flag = False
    page = 30
        
    while True:
        try:
            params['page'] = str(page)
            proxies = {'http' : 'http://' + proxy_data[proxy_idx][1], 'https' : 'http://' + proxy_data[proxy_idx][1]}
            headers = {
                'user-agent' : ua.random
            }
            
            print('request Setting FIN')
            
            res = requests.get(url, headers = headers, proxies = proxies, params=params, timeout=(10,10))
            
            soup = BeautifulSoup(res.text, "html.parser")
            
            last_post_writed_at = soup.select('table.tbl > tbody > tr')[-1].select('td')[2].text
            
            # 내가 찾고자 하는 날짜보다 최근이라면 다음페이지로 이동
            if int(last_post_writed_at[-2:]) > target_date.day:
                print(last_post_writed_at[-2:], target_date.day, "PASSING")
                page+=1
                continue
            
            df, break_flag = get_post(soup, target_date)
            print(page)
            df_lst.append(df)
            
            
            if break_flag:
                break
            
            page+=1
        except Exception as e:
            print(e)
            headers = {'user-agent' : ua.random}
            proxy_idx +=1
            if proxy_idx >= len(proxy_data):
                print('REFRESH PROXY AND RESET IDX')
                proxy_data = get_proxy_list()
                proxy_idx = 0
        
    return  pd.concat(df_lst)


In [6]:
def load_data(file_path, load_date = -1): # 찾으려는 날짜를 특정하지 않으면 어제 게시글 탐색
    y_day = datetime.today() - timedelta(days=1)
    
    if load_date == -1:
    
        load_date = datetime(y_day.year, y_day.month, y_day.day).strftime('%y%m%d')
    
    print(load_date)

    
    path = f'{file_path}_{load_date}.csv'
    
    print(path)

    try:
        df = pd.read_csv(path)
        print('File exist Load FIN!!')
    except:
        df = pd.DataFrame()
        
    return df

In [7]:
def save_daily_data(df, file_path, load_date = -1): # 저장하는 날짜를 특정하지 않으면 어제 게시글로 저장
    y_day = datetime.today() - timedelta(days=1)
    
    if load_date == -1:
        load_date = datetime(y_day.year, y_day.month, y_day.day).strftime('%y%m%d')
    
    path = f'{file_path}_{load_date}.csv'
    
    df.to_csv(path, index=False)
    

In [8]:
def get_proxy_list():
    url = 'http://spys.one/en/free-proxy-list/'
    data = {
        'xpp' : '1',
        'xf1' : '0',
        'xf2' : '0',
        'xf4' : '0',
        'xf5' : '2'
    }

    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }

    r = requests.post(url, data=data, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    result = []

    ports = {}
    script = soup.select_one('body > script')
    for row in script.text.split(';'):
        if '^' in row:
            line = row.split('=')
            ports[line[0]]  = line[1].split('^')[0]

    trs = soup.select('tr[onmouseover]')
    for tr in trs:

        is_http = tr.select_one('a')

        if is_http is not None:
            http = is_http.text
#             print(is_http.text)

        e_ip = tr.select_one('font.spy14')
#         print(e_ip)
        ip = ''

        e_port = tr.select_one('script')
        port = ''
        if e_port is not None:
            re_port = re.compile(r'\(([a-zA-Z0-9]+)\^[a-zA-Z0-9]+\)')
            match = re_port.findall(e_port.text)
            for item in match:
                port = port + ports[item]
        else:
            continue

        if e_ip is not None:
            for item in e_ip.findAll('script'):
                item.extract()
            ip = e_ip.text

        else:
            continue

        tds = tr.select("td")
        is_skip = False
        for td in tds:
            e_pct = td.select_one('font > acronym')
            if e_pct is not None:
                pct = re.sub('([0-9]+)%.*', r'\1', e_pct.text)
                if not pct.isdigit():
                    is_skip = True
            else:
                continue

        if not pct.isdigit():
            continue

        result.append((http,ip + ':' + port, pct))

    result.sort(key = lambda element  : int(element[2]), reverse=True)

    return result

Timestamp('2023-05-11 00:00:00')

In [19]:
dates = ['20230515','20230511', '20230512', '20230513']

In [14]:
# 검색할때 설정해줘야할 것들
for target_date in dates:
    path = './data/offical_page'

    df = load_data(path, target_date)

    if not len(df):
        is_concat = False
    else:
        is_concat = True

    # dc_id = "dnfqq"

    # if len(df) == 0:
    #     y_day = datetime.today() - timedelta(days=1)
    #     y_day_string = str(y_day.year) + "-" + str(y_day.month) + "-" + str(y_day.day)
    #     last_timestamp = pd.to_datetime(y_day_string)
    # else:
    #     last_timestamp = pd.to_datetime(df.sort_values('timestamp', ascending=False)['timestamp'].iloc[0])

    # print(last_timestamp)

    search_data = search_post(pd.to_datetime(target_date))

    if is_concat:
        print('Concat New Post')
        df = pd.concat([pd.DataFrame(search_data), df])
    else:
        print('Frist Crawling')
        df = pd.DataFrame(search_data)

    save_daily_data(df, path, target_date)

20230511
./data/offical_page_20230511.csv


KeyboardInterrupt: 